# Notebook to test the paramters of the LSTM trend model

In [1]:
#imports
import numpy as np
from math import sqrt
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import pandas as pd

from sklearn.utils import shuffle
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dropout
import sklearn
from keras.layers import Activation
from keras.layers import Bidirectional
# from keras.layers import CuDNNLSTM
from keras.optimizers import Adam
from keras.layers import Dense
import keras.utils
from keras.layers import LSTM
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import tensorflow as tf
import seaborn as sn
import matplotlib.pyplot as plt
import seed
import os
tf.get_logger().setLevel('ERROR')

In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

In [3]:
#function to calculate rsi
def rsi(ohlc, period: int = 14) -> pd.Series:
    """See source https://github.com/peerchemist/finta
    and fix https://www.tradingview.com/wiki/Talk:Relative_Strength_Index_(RSI)
    Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements.
    RSI oscillates between zero and 100. Traditionally, and according to Wilder, RSI is considered overbought when above 70 and oversold when below 30.
    Signals can also be generated by looking for divergences, failure swings and centerline crossovers.
    RSI can also be used to identify the general trend."""

    delta = ohlc["Close"].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    _gain = up.ewm(com=(period - 1), min_periods=period).mean()
    _loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()

    RS = _gain / _loss
    return 100 - (100 / (1 + RS))

In [4]:
def get_cols_to_rem_label(n_lag, n_features, label_col):
    to_rem = []
    for i in range(1, n_features+1):
        for j in range(0, n_lag+1):
            if(j!=0 or i != label_col):
                if(j == 0):
                    to_rem.append("var"+str(i)+"(t)")
                else:
                    to_rem.append("var"+str(i)+"(t-"+str(j)+")")
    return to_rem
    
def get_cols_to_rem_feat(n_lag, n_features, label_col):
    #uncomment the below line to only remove the last price
    to_rem = ["var1(t)", "var2(t)"]
    return to_rem

In [5]:
def load_lag_sets(step, lag_features, train_ratio, lag_granularity, lag, dataset_grouped_by, cleaned):
    # type of analyser - TextBlob or vader
    analyser = "vader"
    # analyser = "TextBlob"
    
    #read dataset
    folder = "./../datasets/tweets_prices_volumes_sentiment/"+analyser+"/"+dataset_grouped_by+"_datasets"
    
    if cleaned:
        folder = folder + '/cleaned'
        
    filename = folder+"/final_data_lag_"+lag_granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

    df = pd.read_csv(filename)
    
    #group by datetime
    df = df.groupby('DateTime').agg(lambda x: x.mean())
    
    #calculate change
    df["Change"] = (df["Close"] - df["Close"].shift(1)).astype(float)
    #drop empty
    df = df.dropna(subset=['Change'])
    #max positive change 
    max_change = df["Change"].max()
    #max negative change 
    min_change = df["Change"].min()
    
    #prepare bins
    rnge = max_change - min_change
    bin_size = (max_change - min_change) / 10
    half_range = rnge/2
    bins = np.arange(-1*half_range, half_range, bin_size)
    bins[5] = 0
    bins[0] = float("-inf")
    bins = np.append(bins, float("inf"))
    labels = [0, 1,2,3,4,5,6,7,8,9]
    
    #set bins
    df['Change'] = pd.cut(x=df['Change'], bins=bins, labels=labels, include_lowest=True)

    add_RSI = False
    add_longMAvg = False
    add_shortMAvg = False

    if(add_RSI):
        #calcualte RSI
        RSI = 14
        df['RSI'] = rsi(df, RSI)
        df = df.iloc[RSI:]

    #calcualte moving averages

    if(add_shortMAvg):
        short_window = 9
        df['short_mavg'] = df.rolling(window=short_window)["Close"].mean()

    if(add_longMAvg):
        long_window = 21
        df["long_mavg"] = df.rolling(window=long_window)["Close"].mean()

    if(add_longMAvg):
        df = df.iloc[long_window:]
    elif(add_RSI):
        df = df.iloc[RSI:]
    elif(add_shortMAvg):
        df = df.iloc[short_window:]
        
        
    #keep only wanted columns
    features = ['Change', 'subjectivity', 'polarity','Tweet_vol','Volume_(BTC)'] if analyser == "Textblob" else ['Change', 'Close', 'pos_pol', 'neg_pol', 'Tweet_vol']

    # features = ['Change', 'subjectivity', 'polarity','Tweet_vol','Volume_(BTC)'] if analyser == "Textblob" else ['Change', 'Close', 'compound', 'pos_pol', 'neg_pol', 'neu_pol', 'Tweet_vol','Volume_(BTC)']

    if(add_RSI):
        features.append("RSI")

    if(add_longMAvg):
        features.append("long_mavg")

    if(add_shortMAvg):
        features.append("short_mavg")

    df = df[features]

    #creating copy so that data is not loaded once again
    df_copy = df.copy()
    
    #number of previous records to consider for every example
    n_lag = lag_features
    #number of features
    n_features = len(features)
    #calcualte total_features
    total_features = n_lag*n_features

    if(total_features == 0):
        total_features = n_features
        
    df_reframed = series_to_supervised(df, n_lag, 1)
    df_reframed =  df_reframed.reset_index()
    df_reframed = df_reframed.drop(['DateTime'], axis=1)
    
    #shuffle
    np.random.seed(1)
    for j in range(0, step+1):
        df_reframed = shuffle(df_reframed)
    
    #divide df into train and test
    data_len = len(df_reframed)
    train_size = int(data_len*train_ratio)

    train = df_reframed.iloc[:train_size]
    test = df_reframed.iloc[train_size:]
    
    train_y = train["var1(t)"].values
    test_y = test["var1(t)"].values

#     train_y = train_y.reshape(len(train_y), 1)
#     test_y = test_y.reshape(len(test_y), 1)
    
    train_reframed = train
    test_reframed = test
    
    train_reframed = train_reframed.drop(get_cols_to_rem_feat(n_lag, n_features, 1), axis=1)
    test_reframed = test_reframed.drop(get_cols_to_rem_feat(n_lag, n_features, 1), axis=1)
    
    #normalise features
    xscaler = MinMaxScaler(feature_range=(0, 1))
    train_reframed = xscaler.fit_transform(train_reframed)
    test_reframed = xscaler.transform(test_reframed)

    train = train_reframed
    test = test_reframed
    train_labels = train_y
    test_labels = test_y

    train = train_reframed
    test = test_reframed
    
    #remove the last set of values(data of time to be predicted)
    train = train[:, :total_features]
    test = test[:, :total_features]

    #keep only prices array
    train_X, train_y = train[:, :total_features], train_y
    test_X, test_y = test[:, :total_features], test_y

    # reshape input to be 3D [samples, timesteps, features]
    if(n_lag > 0):
        train_X = train_X.reshape((train_X.shape[0], n_lag, n_features))
        test_X = test_X.reshape((test_X.shape[0], n_lag, n_features))
    else:
        train_X = train_X.reshape((train_X.shape[0], 1, n_features-1))
        test_X = test_X.reshape((test_X.shape[0], 1, n_features-1))

    train_y = keras.utils.to_categorical(train_y, 10)
    
    return train_X, test_X, train_y, test_y, len(features), df, train_size

In [6]:
def create_model_test(epochs, neurons, batch_size, layers, train_X, test_X, train_y, test_y, lag_features, features, df, train_size):
    global n_lag
    global n_features
    
    np.random.seed(1)
    tf.random.set_seed(1)

    # design network
    model = Sequential()
    dropout = 0.25
    activ_func = "linear"
    
    return_seq = layers > 1

    model.add(LSTM(neurons, return_sequences=return_seq, input_shape=(train_X.shape[1], train_X.shape[2]), activation=activ_func))
    model.add(Dropout(dropout))
    
    for i in range(1, layers):
        ret_seq = i != (layers-1)
        model.add(LSTM(neurons, return_sequences=ret_seq, activation=activ_func))
        model.add(Dropout(dropout))

    model.add(Dense(10, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit network
    history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,validation_split=0.2)
    
    if(lag_features > 0):
        test_X = test_X.reshape((test_X.shape[0], lag_features, features))
    else:
        test_X = test_X.reshape((test_X.shape[0], 1, features-1))

    pred = model.predict(test_X)

    if(lag_features > 0):
        test_X = test_X.reshape((test_X.shape[0], lag_features* features,))
    else:
        test_X = test_X.reshape((test_X.shape[0], features-1,))
        
    pred = np.argmax(pred, axis=1)
    
    report = sklearn.metrics.classification_report(test_y, pred, zero_division=0,output_dict=True)
    
    #obtain f1-scores for classes
    f1 = np.zeros(10)
    for i in range(0,10):
        f1[i] = report[str(i)]['f1-score'] if str(i) in report else 0
        
    #return accuracy and f1
    return report['accuracy'], f1

In [13]:
def implement(lag_granularity, lag, dataset_grouped_by, cleaned):
    #get filename
    filename = 'results/lstm_multiclass/lstm_multiclass_groupedby_'+dataset_grouped_by+"_lag_"+lag_granularity+"_"+str(lag)

    if cleaned:
        filename = filename + '_cleaned'

    full_filename = filename+".csv"

    columns = ["lag", "epochs", "batch_size", "neurons", "layers", "split", "accuracy", "f1-0", "f1-1", "f1-2", "f1-3", "f1-4", "f1-5", "f1-6", "f1-7", "f1-8", "f1-9"]
    try:
        results = pd.read_csv(full_filename)
    except:
        results = pd.DataFrame(columns=columns)
    lags = [3]
    train_ratio = 0.85
    
    for lag_features in lags:

        epochs = [50, 200, 500, 1000, 2000]
        neurons = [32, 128, 256]
        layers = [1, 2, 3]
        batch_sizes = [5, 20, 50, 80]
        
#         epochs = [50]
#         neurons = [32]
#         layers = [1]
#         batch_sizes = [500]

        for e in epochs:
            for n in neurons:
                for l in layers:
                    for b in  batch_sizes:
                        accuracies = []
                        f1_0 = []
                        f1_1 = []
                        f1_2 = []
                        f1_3 = []
                        f1_4 = []
                        f1_5 = []
                        f1_6 = []
                        f1_7 = []
                        f1_8 = []
                        f1_9 = []
                        print("Testing model: lag:", lag_features, ", epochs:", e, ", neurons:", n, ", layers:", l, ", batch_size:", b)
                        
                        for i in range (0,5):
                            train_X, test_X, train_y, test_y, features, df, train_size = load_lag_sets(i, lag_features, train_ratio, lag_granularity, lag, dataset_grouped_by, cleaned)
                            acc, f1 = create_model_test(e, n, b, l, train_X, test_X, train_y, test_y, lag_features, features, df, train_size)
                            accuracies.append(acc)
                            f1_0.append(f1[0])
                            f1_1.append(f1[1])
                            f1_2.append(f1[2])
                            f1_3.append(f1[3])
                            f1_4.append(f1[4])
                            f1_5.append(f1[5])
                            f1_6.append(f1[6])
                            f1_7.append(f1[7])
                            f1_8.append(f1[8])
                            f1_9.append(f1[9])
                            
                        
                        accuracies = np.array(accuracies)
                        f1_0 = np.array(f1_0)
                        f1_1 = np.array(f1_1)
                        f1_2 = np.array(f1_2)
                        f1_3 = np.array(f1_3)
                        f1_4 = np.array(f1_4)
                        f1_5 = np.array(f1_5)
                        f1_6 = np.array(f1_6)
                        f1_7 = np.array(f1_7)
                        f1_8 = np.array(f1_8)
                        f1_9 = np.array(f1_9)
                        
                        mean_acc =accuracies.mean()
                        min_acc =accuracies.min()
                        max_acc =accuracies.max()
                        diff_acc = max_acc - min_acc
                        
                        mean_f1_0 = f1_0.mean();
                        mean_f1_1 = f1_1.mean();
                        mean_f1_2 = f1_2.mean();
                        mean_f1_3 = f1_3.mean();
                        mean_f1_4 = f1_4.mean();
                        mean_f1_5 = f1_5.mean();
                        mean_f1_6 = f1_6.mean();
                        mean_f1_7 = f1_7.mean();
                        mean_f1_8 = f1_8.mean();
                        mean_f1_9 = f1_9.mean();
                        
                        results = results.append({"lag": lag_features, "epochs": e, "batch_size": b, "neurons":n, "layers":l, "split": train_ratio, "mean_acc": mean_acc, "min_acc": min_acc, "max_acc": max_acc, "diff_acc":diff_acc, "f1-0": mean_f1_0, "f1-1": mean_f1_1, "f1-2": mean_f1_2, "f1-3": mean_f1_3, "f1-4": mean_f1_4, "f1-5": mean_f1_5, "f1-6": mean_f1_6, "f1-7": mean_f1_7, "f1-8": mean_f1_8, "f1-9": mean_f1_9}, ignore_index=True)
                
    return results, full_filename
        

In [14]:
#lag granularity - days or hours
lag_granularity = "days"
#lag value
lag = 1
#dataset grouped type - day or hour
dataset_grouped_by = "day"
#cleaned
cleaned = True

In [ ]:
results, full_filename = implement(lag_granularity, lag, dataset_grouped_by, cleaned)

Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 1 , batch_size: 5
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 1 , batch_size: 20
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 1 , batch_size: 50
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 1 , batch_size: 80
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 2 , batch_size: 5
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 2 , batch_size: 20
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 2 , batch_size: 50
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 2 , batch_size: 80
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 3 , batch_size: 5
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 3 , batch_size: 20
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 3 , batch_size: 50
Testing model: lag: 3 , epochs: 50 , neurons: 32 , layers: 3 , batch_size: 80
Testing model: lag: 3 , epochs: 50 , neurons: 128 , layers: 1 , bat

Testing model: lag: 3 , epochs: 500 , neurons: 256 , layers: 3 , batch_size: 5
Testing model: lag: 3 , epochs: 500 , neurons: 256 , layers: 3 , batch_size: 20
Testing model: lag: 3 , epochs: 500 , neurons: 256 , layers: 3 , batch_size: 50
Testing model: lag: 3 , epochs: 500 , neurons: 256 , layers: 3 , batch_size: 80
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 1 , batch_size: 5
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 1 , batch_size: 20
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 1 , batch_size: 50
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 1 , batch_size: 80
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 2 , batch_size: 5
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 2 , batch_size: 20
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 2 , batch_size: 50
Testing model: lag: 3 , epochs: 1000 , neurons: 32 , layers: 2 , batch_size: 80
Testing model: lag: 3 , epochs: 1000 , neur

In [11]:
results

,lag,epochs,batch_size,neurons,layers,split,accuracy,f1-0,f1-1,f1-2,...,f1-4,f1-5,f1-6,f1-7,f1-8,f1-9,diff_acc,max_acc,mean_acc,min_acc
0,1.0,50.0,5.0,32.0,1.0,0.85,NaN,0.0,0.0,0.000000,...,0.063333,0.603266,0.000000,0.0,0.0,0.0,0.102941,0.500000,0.435294,0.397059
1,1.0,50.0,20.0,32.0,1.0,0.85,NaN,0.0,0.0,0.000000,...,0.101333,0.544526,0.000000,0.0,0.0,0.0,0.117647,0.500000,0.426471,0.382353
2,1.0,50.0,50.0,32.0,1.0,0.85,NaN,0.0,0.0,0.000000,...,0.101099,0.488782,0.000000,0.0,0.0,0.0,0.176471,0.514706,0.420588,0.338235
3,1.0,50.0,80.0,32.0,1.0,0.85,NaN,0.0,0.0,0.000000,...,0.083721,0.598782,0.000000,0.0,0.0,0.0,0.117647,0.514706,0.444118,0.397059
4,1.0,50.0,5.0,32.0,2.0,0.85,NaN,0.0,0.0,0.000000,...,0.000000,0.615513,0.000000,0.0,0.0,0.0,0.102941,0.500000,0.444118,0.397059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,1.0,2000.0,80.0,256.0,2.0,0.85,NaN,0.0,0.0,0.200000,...,0.414110,0.527565,0.190000,0.0,0.0,0.0,0.161765,0.514706,0.420588,0.352941
176,1.0,2000.0,5.0,256.0,3.0,0.85,NaN,0.0,0.0,0.000000,...,0.446486,0.491932,0.166667,0.0,0.0,0.0,0.176471,0.514706,0.417647,0.338235
177,1.0,2000.0,20.0,256.0,3.0,0.85,NaN,0.0,0.0,0.000000,...,0.415333,0.465263,0.175253,0.0,0.0,0.0,0.132353,0.441176,0.388235,0.308824
178,1.0,2000.0,50.0,256.0,3.0,0.85,NaN,0.0,0.0,0.200000,...,0.390143,0.468617,0.128889,0.0,0.0,0.0,0.176471,0.455882,0.382353,0.279412


In [12]:
results.to_csv(full_filename, index=False)